In [1]:
import pandas as pd
import sourmash

In [2]:
# ICTV VMR lineages
vmr_lineages = 'from-farm/vmr_MSL38_v1.taxonomy.csv'
vmr = pd.read_csv(vmr_lineages)
# force use GCA version of identifiers 
vmr['GCA_ident'] = vmr['ident'].str.replace('GCF', 'GCA')
# now truncate the version so we match across assembly versions
vmr['GCA_ident'] = vmr['GCA_ident'].str.split('.', expand=True)[0]

# homogenize VMR lineage format by removing 'Viruses' and replacing ',' with ';'
vmr['lineage'] = vmr['lineage'].str.split('Viruses,', expand=True)[1].str.rsplit(',', n=1, expand=True)[0]
vmr['lineage'] = vmr['lineage'].str.replace(',', ';')
vmr.head()

,ident,genbank_ident,refseq_ident,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,...,subfamily,genus,subgenus,species,name,lineage,exemplar_or_additional,gene_accs,protein_accs,GCA_ident
0,GCF_003950175.1,MK064563,NC_048128,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Alphalipothrixvirus,NaN,Alphalipothrixvirus SBFV2,Sulfolobales Beppu filamentous virus 2,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_048128.1,"YP_009817896.1,YP_009817897.1,YP_009817898.1,Y...",GCA_003950175
1,GCF_003441495.1,MH447526,NC_048037,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Alphalipothrixvirus,NaN,Alphalipothrixvirus SFV1,Sulfolobus filamentous virus 1,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_048037.1,"YP_009808111.1,YP_009808112.1,YP_009808113.1,Y...",GCA_003441495
2,GCF_000878935.1,AM087120,NC_010155,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 3,Acidianus filamentous virus 3,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010155.1,"YP_001604343.1,YP_001604344.1,YP_001604345.1,Y...",GCA_000878935
3,GCF_000871385.1,AM087121,NC_010152,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 6,Acidianus filamentous virus 6,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010152.1,"YP_001604159.1,YP_001604160.1,YP_001604161.1,Y...",GCA_000871385
4,GCF_000872245.1,AM087122,NC_010153,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 7,Acidianus filamentous virus 7,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010153.1,"YP_001604225.1,YP_001604226.1,YP_001604227.1,Y...",GCA_000872245


In [3]:
vmr_idents = vmr['GCA_ident'].tolist()
vmr_idents[:5]

['GCA_003950175',
 'GCA_003441495',
 'GCA_000878935',
 'GCA_000871385',
 'GCA_000872245']

In [4]:
k31_sc200_csv = 'from-farm/genbank.2023-05.viral-x-vmr_MSL38_v1.dna-k31-sc200.fastmultigather.with-lineages.csv'
k31_sc10_csv = 'from-farm/genbank.2023-05.viral-x-vmr_MSL38_v1.dna-k31-sc10.fastmultigather.with-lineages.csv'
k21_sc200_csv = 'from-farm/genbank.2023-05.viral-x-vmr_MSL38_v1.dna-k21-sc200.fastmultigather.with-lineages.csv'
k21_sc10_csv = 'from-farm/genbank.2023-05.viral-x-vmr_MSL38_v1.dna-k21-sc10.fastmultigather.with-lineages.csv'

k31sc10 = pd.read_csv(k31_sc10_csv)
k31sc200 = pd.read_csv(k31_sc200_csv)
k21sc10 = pd.read_csv(k21_sc10_csv)
k21sc200 = pd.read_csv(k21_sc200_csv)

In [5]:
k31sc10.shape

(134659, 7)

In [6]:
# number actually annotated
k31sc10['query_name'].nunique()

45535

In [7]:
k31sc10['match_name'].nunique()

12544

In [8]:
k21sc10.shape

(340391, 7)

In [9]:
k21sc10['query_name'].nunique()

56378

In [10]:
k21sc200['query_name'].nunique()

45879

In [11]:
k31sc10['query_ident'] = k31sc10['query_name'].str.split(' ', n=1, expand=True)[0].str.split('.', expand=True)[0].str.replace('GCF', 'GCA')
k31sc10['match_ident'] = k31sc10['match_name'].str.split(' ', n=1, expand=True)[0].str.split('.', expand=True)[0].str.replace('GCF', 'GCA')

k31sc10.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage,query_ident,match_ident
0,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000856265.1 Potato spindle tuber viroid,9bd95782c87c981310bde7cc478dde67,0.875000,210,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...,GCA_000889915,GCA_000856265
1,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000855405.1 Tomato chlorotic dwarf viroid,44458a1b4cfc26470fea9118027548ec,0.000000,30,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Tomato c...,GCA_000889915,GCA_000855405
2,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.365517,530,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_002819625
3,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_000837765.1 Porcine circovirus 1,ce14d1cfe5ae6c09de4bfb6de490efb9,0.011976,60,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_000837765
4,GCA_004040595.1 Porcine circovirus 2,b4bcdaeae74a0545b8666fb21e44ac6d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.627586,910,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004040595,GCA_002819625


In [12]:
k31sc10.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage,query_ident,match_ident
0,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000856265.1 Potato spindle tuber viroid,9bd95782c87c981310bde7cc478dde67,0.875000,210,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...,GCA_000889915,GCA_000856265
1,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000855405.1 Tomato chlorotic dwarf viroid,44458a1b4cfc26470fea9118027548ec,0.000000,30,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Tomato c...,GCA_000889915,GCA_000855405
2,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.365517,530,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_002819625
3,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_000837765.1 Porcine circovirus 1,ce14d1cfe5ae6c09de4bfb6de490efb9,0.011976,60,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_000837765
4,GCA_004040595.1 Porcine circovirus 2,b4bcdaeae74a0545b8666fb21e44ac6d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.627586,910,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004040595,GCA_002819625


In [13]:
# since we have gather results, keep first match only for now
k31sc10g = k31sc10.groupby('query_ident').nth(0).reset_index()

In [14]:
k31sc10g.head()

,index,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage,query_ident,match_ident
0,0,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000856265.1 Potato spindle tuber viroid,9bd95782c87c981310bde7cc478dde67,0.875000,210,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...,GCA_000889915,GCA_000856265
1,2,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.365517,530,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_002819625
2,4,GCA_004040595.1 Porcine circovirus 2,b4bcdaeae74a0545b8666fb21e44ac6d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.627586,910,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004040595,GCA_002819625
3,5,GCA_004074215.1 Porcine circovirus 2,8c16258e8d1a1d94f96b90c2366db946,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.606897,880,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004074215,GCA_002819625
4,6,GCA_004052835.1 Porcine circovirus 2,4ce0555747f6d6f45ac5436d5343e44d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.427586,620,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004052835,GCA_002819625


In [15]:
k31sc10g.shape

(45535, 10)

### First, let's look at just the genomes in ICTV-VMR and ensure we got good lineages 
(should be 100% match, identical genome in database)

In [16]:
query_in_vmr = k31sc10g[k31sc10g['query_ident'].isin(vmr_idents)]
query_in_vmr.shape

(12500, 10)

= a few vmr accessions did not have matches... let's dig a little

In [17]:
non_matching = query_in_vmr[~(query_in_vmr['query_ident'] == query_in_vmr['match_ident'])]
non_matching[['query_name', 'match_name', 'lineage']]

,query_name,match_name,lineage
13427,GCA_023120215.1 Simian adenovirus 4,GCF_000846705.1 Simian adenovirus 3,Varidnaviria;;Bamfordvirae;;Preplasmiviricota;...
17967,GCA_002756135.1 Listeria phage LMTA-34,GCF_000923595.1 Listeria phage LMSP-25,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...
20649,GCA_002829985.1 Una virus,GCF_002889395.1 Una virus,Riboviria;;Orthornavirae;;Kitrinoviricota;;Als...
25094,GCA_002827965.1 Ugandan cassava brown streak v...,GCF_000888855.1 Ugandan cassava brown streak v...,Riboviria;;Orthornavirae;;Pisuviricota;;Stelpa...
27017,GCA_002623205.1 Propionibacterium phage PHL194M00,GCF_002623185.1 Propionibacterium phage PHL163M00,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...
29670,GCA_003972205.1 Extra small virus,GCF_003972145.1 Extra small virus,Riboviria;;;;;;;;;;Sarthroviridae;;Macronoviru...
30027,GCA_004786415.1 Bussuquara virus,GCF_000872985.1 Aroa virus,Riboviria;;Orthornavirae;;Kitrinoviricota;;Fla...


In [18]:
non_matching['lineage'].tolist()

['Varidnaviria;;Bamfordvirae;;Preplasmiviricota;;Tectiliviricetes;;Rowavirales;;Adenoviridae;;Mastadenovirus;;Simian mastadenovirus A;simian adenovirus 3',
 'Duplodnaviria;;Heunggongvirae;;Uroviricota;;Caudoviricetes;;;;Herelleviridae;Jasinkavirinae;Pecentumvirus;;Pecentumvirus LMSP25;Listeria phage LMSP-25',
 'Riboviria;;Orthornavirae;;Kitrinoviricota;;Alsuviricetes;;Martellivirales;;Togaviridae;;Alphavirus;;Una virus;Una virus',
 'Riboviria;;Orthornavirae;;Pisuviricota;;Stelpaviricetes;;Patatavirales;;Potyviridae;;Ipomovirus;;Ugandan cassava brown streak virus;Ugandan cassava brown streak virus',
 'Duplodnaviria;;Heunggongvirae;;Uroviricota;;Caudoviricetes;;;;;;Pahexavirus;;Pahexavirus PHL163M00;Propionibacterium phage PHL163M00',
 'Riboviria;;;;;;;;;;Sarthroviridae;;Macronovirus;;Macrobrachium satellite virus 1;extra small virus',
 'Riboviria;;Orthornavirae;;Kitrinoviricota;;Flasuviricetes;;Amarillovirales;;Flaviviridae;;Orthoflavivirus;;Orthoflavivirus aroaense;Aroa virus']

In [19]:
matched_queries = k31sc10g['query_ident'].tolist()
vmr_missing = [x for x in vmr_idents if x not in matched_queries and 'GCA' in x]
len(vmr_missing)

17

In [20]:
vmr_missing

['GCA_004768525',
 'GCA_003052045',
 'GCA_003052025',
 'GCA_000498655',
 'GCA_001611975',
 'GCA_002951655',
 'GCA_023685825',
 'GCA_017916365',
 'GCA_023596645',
 'GCA_023613675',
 'GCA_003495705',
 'GCA_000046705',
 'GCA_000022105',
 'GCA_000018145',
 'GCA_000021865',
 'GCA_001633165',
 'GCA_000011965']

In [21]:
 vmr[vmr['GCA_ident'].isin(vmr_missing)]

,ident,genbank_ident,refseq_ident,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,...,subfamily,genus,subgenus,species,name,lineage,exemplar_or_additional,gene_accs,protein_accs,GCA_ident
1410,GCF_004768525.1,CP038625,NaN,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Finvirus,NaN,Finvirus FIN13,Arsenophonus nasoniae strain FIN plasmid pArsF...,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NZ_CP038613.1,NZ_CP038614.1,NZ_CP038615.1,NZ_C...","WP_000135199.1,WP_000429386.1,WP_001181005.1,W...",GCA_004768525
1460,GCF_003052045.1,CP023680,NaN,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Plazymidvirus,NaN,Plazymidvirus ZM36,Zymomonas mobilis subsp. mobilis strain ZM4 su...,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NZ_CP023677.1,NZ_CP023678.1,NZ_CP023679.1,NZ_C...","WP_000098614.1,WP_000168720.1,WP_000275334.1,W...",GCA_003052045
1461,GCF_003052025.1,CP023686,NaN,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Plazymidvirus,NaN,Plazymidvirus ZM41,Zymomonas mobilis subsp. mobilis plasmid pZM41,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NZ_CP023682.1,NZ_CP023683.1,NZ_CP023684.1,NZ_C...","WP_000098614.1,WP_000168720.1,WP_000275334.1,W...",GCA_003052025
1462,GCF_000498655.1,CP006891,NC_022901,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Plazymidvirus,NaN,Plazymidvirus ZM401,Zymomonas mobilis subsp. mobilis ZM4 plasmid p...,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NC_022900.1,NC_022901.1,NC_022913.1,NC_022902....","WP_000122735.1,WP_000168720.1,WP_007403889.1,W...",GCA_000498655
1468,GCF_001611975.1,CP014526,NaN,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Sanguivirus,NaN,Sanguivirus H5569V,Haematospirillum jordaniae strain H5569 plasmi...,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NZ_CP014525.1,NZ_CP014526.1,NZ_CP014527.1","WP_066131749.1,WP_066131751.1,WP_066131753.1,W...",GCA_001611975
1481,GCF_002951655.1,CP019275,NaN,Viruses,Duplodnaviria,NaN,Heunggongvirae,NaN,Uroviricota,NaN,...,NaN,Wadgaonvirus,NaN,Wadgaonvirus V13P477T2,Escherichia coli strain 13P477T plasmid p13P47...,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...,E,"NZ_CP019273.1,NZ_CP019274.1,NZ_CP021100.1,NZ_C...","WP_000002294.1,WP_000002542.1,WP_000002778.1,W...",GCA_002951655
6068,GCA_023685825.1,M81103,NC_003822,Viruses,Monodnaviria,NaN,Shotokuvirae,NaN,Cressdnaviricota,NaN,...,NaN,Mastrevirus,NaN,Tobacco yellow dwarf virus,tobacco yellow dwarf virus,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,E,"DAHJTY010000001.1,DAHJTY010000002.1,DAHJTY0100...","HCD7521966.1,HCD7521967.1,HCD7521968.1,HCD7521...",GCA_023685825
6858,GCF_017916365.1,D00344,NC_011620,Viruses,Riboviria,NaN,Orthornavirae,NaN,Kitrinoviricota,NaN,...,NaN,Potexvirus,NaN,Potato virus X,potato virus X,Riboviria;;Orthornavirae;;Kitrinoviricota;;Als...,E,"NZ_JAFMSU010000001.1,NZ_JAFMSU010000002.1,NZ_J...","WP_022821757.1,WP_048919288.1,WP_048919295.1,W...",GCA_017916365
9956,GCA_023596645.1,M96262,NC_043487,Viruses,Riboviria,NaN,Orthornavirae,NaN,Pisuviricota,NaN,...,Variarterivirinae,Betaarterivirus,Eurpobartevirus,Betaarterivirus suid 1,porcine reproductive and respiratory syndrome ...,Riboviria;;Orthornavirae;;Pisuviricota;;Pisoni...,E,"DAHING010000001.1,DAHING010000002.1,DAHING0100...","HCD3982833.1,HCD3982834.1,HCD3982835.1,HCD3982...",GCA_023596645
11356,GCA_023613675.1,M89923,NC_008017,Viruses,Riboviria,NaN,Pararnavirae,NaN,Artverviricota,NaN,...,NaN,Badnavirus,NaN,Badnavirus deltasacchari,sugarcane bacilliform MO virus,Riboviria;;Pararnavirae;;Artverviricota;;Revtr...,E,"DAHJBQ010000001.1,DAHJBQ010000002.1,DAHJBQ0100...","HCD5405667.1,HCD5405668.1,HCD5405669.1,HCD5405...",GCA_023613675


I looked up 2 of these 'missing' / unannotated genomes, and they were 5kb and 649bp respectively, suggesting that they were likely too small for confident calls. Did I use threshold = 0 with these?

#### Read in lengths file, summarize length info

In [22]:
len_csv = 'from-farm/genbank.2023-05.viral.fasta-lengths.csv'
ln = pd.read_csv(len_csv)
ln['GCA_ident'] = ln['name'].str.split(' ', n=1, expand=True)[0].str.replace('GCF_', 'GCA_').str.split('.', n=1, expand=True)[0]
ln.head()

,name,genome_filename,protein_filename,basepair_count,amino_acid_count,GCA_ident
0,GCA_000839185.1 Cowpox virus,genbank/genomes/GCA_000839185.1_genomic.fna.gz,genbank/proteomes/GCA_000839185.1_protein.faa.gz,224499,69582.0,GCA_000839185
1,GCA_003971385.1 Cowpox virus,genbank/genomes/GCA_003971385.1_genomic.fna.gz,genbank/proteomes/GCA_003971385.1_protein.faa.gz,204218,54684.0,GCA_003971385
2,GCA_003971405.1 Cowpox virus,genbank/genomes/GCA_003971405.1_genomic.fna.gz,genbank/proteomes/GCA_003971405.1_protein.faa.gz,203925,53613.0,GCA_003971405
3,GCA_004025355.1 Cowpox virus,genbank/genomes/GCA_004025355.1_genomic.fna.gz,genbank/proteomes/GCA_004025355.1_protein.faa.gz,214880,55941.0,GCA_004025355
4,GCA_004025395.1 Cowpox virus,genbank/genomes/GCA_004025395.1_genomic.fna.gz,genbank/proteomes/GCA_004025395.1_protein.faa.gz,228369,58448.0,GCA_004025395


In [23]:
vmr_missing

['GCA_004768525',
 'GCA_003052045',
 'GCA_003052025',
 'GCA_000498655',
 'GCA_001611975',
 'GCA_002951655',
 'GCA_023685825',
 'GCA_017916365',
 'GCA_023596645',
 'GCA_023613675',
 'GCA_003495705',
 'GCA_000046705',
 'GCA_000022105',
 'GCA_000018145',
 'GCA_000021865',
 'GCA_001633165',
 'GCA_000011965']

In [24]:
vmissingk31 = vmr[vmr['GCA_ident'].isin(vmr_missing)]
vmissingk31.shape

(17, 25)

In [25]:
vmissingk31_len = pd.merge(vmissingk31, ln, on='GCA_ident', how='left')
vmissingk31_len.shape

(17, 30)

In [26]:
vmissingk31_len['basepair_count'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: basepair_count, dtype: float64

ok - so **likely these are missing due to download issues/errors. -- I did not have the fast file available to count length**
Since I did a fair bit of manipulation to exclude supressed records/
build genomes where needed for VMR, I should just go find these there

In [27]:
k21sc10.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage
0,GCA_000889915.1 Rubber viroid India/2009,556609d66345ece1a6783599e2365179,GCF_000856265.1 Potato spindle tuber viroid,d43ee9f62bb9f7a6a8c89bf3b8032e55,0.950000,380,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...
1,GCA_000889915.1 Rubber viroid India/2009,556609d66345ece1a6783599e2365179,GCF_000852825.1 Iresine viroid 1,2a60d4cdd2197eae67d144460d70cdfb,0.000000,30,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Iresine ...
2,GCA_004062655.1 Porcine circovirus 2,94d4d702904af5c1ff378728a972a4f3,GCF_002819625.1 Porcine circovirus 2,5ea3286cff69525e2ef330b03cc10c48,0.594872,1160,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...
3,GCA_004062655.1 Porcine circovirus 2,94d4d702904af5c1ff378728a972a4f3,GCF_000837765.1 Porcine circovirus 1,189f76184f1d1335d3315829746d6c41,0.027322,170,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...
4,GCA_004062655.1 Porcine circovirus 2,94d4d702904af5c1ff378728a972a4f3,GCF_029886425.1 WigFec circovirus 1,0343ec260cee3d4b700241d6f54668e8,0.000000,10,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...


In [28]:
k21sc10['query_ident'] = k21sc10['query_name'].str.split(' ', n=1, expand=True)[0].str.split('.', expand=True)[0].str.replace('GCF', 'GCA')
k21sc10['match_ident'] = k21sc10['match_name'].str.split(' ', n=1, expand=True)[0].str.split('.', expand=True)[0].str.replace('GCF', 'GCA')
# since we have gather results, keep first match only
k21sc10g = k21sc10.groupby('query_ident').nth(0).reset_index()
k21sc10g.shape

(56378, 10)

In [29]:
k21_query_in_vmr = k21sc10g[k21sc10g['query_ident'].isin(vmr_idents)]
k21_query_in_vmr.shape

(12500, 10)

In [30]:
# False Positives ("incorrect" annotation)
non_matchingk21 = k21_query_in_vmr[~(k21_query_in_vmr['query_ident'] == k21_query_in_vmr['match_ident'])]
non_matchingk21[['query_name', 'match_name', 'lineage']]

,query_name,match_name,lineage
4640,GCA_002868415.1 Heterobasidion partitivirus 7,GCF_002378515.1 Heterobasidion partitivirus 7,Riboviria;;Orthornavirae;;Pisuviricota;;Duplop...
13319,GCA_002827965.1 Ugandan cassava brown streak v...,GCF_000888855.1 Ugandan cassava brown streak v...,Riboviria;;Orthornavirae;;Pisuviricota;;Stelpa...
26100,GCA_002829985.1 Una virus,GCF_002889395.1 Una virus,Riboviria;;Orthornavirae;;Kitrinoviricota;;Als...
28017,GCA_004786415.1 Bussuquara virus,GCF_000872985.1 Aroa virus,Riboviria;;Orthornavirae;;Kitrinoviricota;;Fla...
31630,GCA_002603105.1 Vibrio phage Vf12,GCF_000845325.1 Villovirus Vf33,Monodnaviria;;Loebvirae;;Hofneiviricota;;Faser...
34169,GCA_023120215.1 Simian adenovirus 4,GCF_000846705.1 Simian adenovirus 3,Varidnaviria;;Bamfordvirae;;Preplasmiviricota;...
56346,GCA_002756135.1 Listeria phage LMTA-34,GCF_000923595.1 Listeria phage LMSP-25,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Ca...


The top 2 look like equally good matches where we picked the 'wrong' genome == lineage is identical

In [31]:
non_matchingk21.shape

(7, 10)

In [32]:
k21matched_queries = k21sc10g['query_ident'].tolist()
k21vmr_missing = [x for x in vmr_idents if x not in k21matched_queries and 'GCA' in x]
len(k21vmr_missing)

17

### Now, let's look at non-VMR genomes and assess our ICTV annotations

In [33]:
new_annots = k31sc10g[~k31sc10g['query_ident'].isin(vmr_idents)]
new_annots.head()

,index,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage,query_ident,match_ident
0,0,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000856265.1 Potato spindle tuber viroid,9bd95782c87c981310bde7cc478dde67,0.875000,210,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...,GCA_000889915,GCA_000856265
1,2,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.365517,530,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_002819625
2,4,GCA_004040595.1 Porcine circovirus 2,b4bcdaeae74a0545b8666fb21e44ac6d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.627586,910,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004040595,GCA_002819625
3,5,GCA_004074215.1 Porcine circovirus 2,8c16258e8d1a1d94f96b90c2366db946,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.606897,880,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004074215,GCA_002819625
4,6,GCA_004052835.1 Porcine circovirus 2,4ce0555747f6d6f45ac5436d5343e44d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.427586,620,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004052835,GCA_002819625


In [34]:
tax_ranks = [
        'realm', 'subrealm', 'kingdom', 'subkingdom', 'phylum', 'subphylum', 
        'class', 'subclass', 'order', 'suborder', 'family', 'subfamily', 
        'genus', 'subgenus', 'species'
    ]

def split_lineage_column(df, lineage_column='lineage', ranks=tax_ranks):
    # Split the lineage column
    split_df = df[lineage_column].str.split(';', expand=True)

    # Assign the new column names
    for i, col_name in enumerate(tax_ranks):
        if i < len(split_df.columns):
            df[col_name] = split_df[i]
        else:
            df[col_name] = None  # Assign None if there are fewer elements than tax ranks

    return df

In [35]:
new_ann = split_lineage_column(new_annots)
new_ann.columns

/var/folders/pv/_f7m99kd33782sdc79lfqc300000gn/T/ipykernel_82141/3314065020.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = split_df[i]
/var/folders/pv/_f7m99kd33782sdc79lfqc300000gn/T/ipykernel_82141/3314065020.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = split_df[i]
/var/folders/pv/_f7m99kd33782sdc79lfqc300000gn/T/ipykernel_82141/3314065020.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Index(['index', 'query_name', 'query_md5', 'match_name', 'match_md5',
       'f_match_query', 'intersect_bp', 'lineage', 'query_ident',
       'match_ident', 'realm', 'subrealm', 'kingdom', 'subkingdom', 'phylum',
       'subphylum', 'class', 'subclass', 'order', 'suborder', 'family',
       'subfamily', 'genus', 'subgenus', 'species'],
      dtype='object')

In [36]:
new_ann.head()

,index,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,lineage,query_ident,match_ident,...,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species
0,0,GCA_000889915.1 Rubber viroid India/2009,425cbeaea65f50ce36143dbf83b23689,GCF_000856265.1 Potato spindle tuber viroid,9bd95782c87c981310bde7cc478dde67,0.875000,210,;;;;;;;;;;Pospiviroidae;;Pospiviroid;;Potato s...,GCA_000889915,GCA_000856265,...,,,,,,Pospiviroidae,,Pospiviroid,,Potato spindle tuber viroid
1,2,GCA_004061035.1 Porcine circovirus 2,b8c1d8077a5282a8b3e9531dc5de45d2,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.365517,530,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004061035,GCA_002819625,...,,Arfiviricetes,,Cirlivirales,,Circoviridae,,Circovirus,,Circovirus porcine2
2,4,GCA_004040595.1 Porcine circovirus 2,b4bcdaeae74a0545b8666fb21e44ac6d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.627586,910,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004040595,GCA_002819625,...,,Arfiviricetes,,Cirlivirales,,Circoviridae,,Circovirus,,Circovirus porcine2
3,5,GCA_004074215.1 Porcine circovirus 2,8c16258e8d1a1d94f96b90c2366db946,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.606897,880,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004074215,GCA_002819625,...,,Arfiviricetes,,Cirlivirales,,Circoviridae,,Circovirus,,Circovirus porcine2
4,6,GCA_004052835.1 Porcine circovirus 2,4ce0555747f6d6f45ac5436d5343e44d,GCF_002819625.1 Porcine circovirus 2,49e86df4188300b48263a1fb78bc3a15,0.427586,620,Monodnaviria;;Shotokuvirae;;Cressdnaviricota;;...,GCA_004052835,GCA_002819625,...,,Arfiviricetes,,Cirlivirales,,Circoviridae,,Circovirus,,Circovirus porcine2


In [37]:
new_ann['species'].nunique()

3550